In [1]:
import pandas as pd
import numpy as np

from aif360.algorithms.preprocessing import DisparateImpactRemover,Reweighing,LFR
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric

from sklearn.ensemble import RandomForestClassifier
from aif360.datasets import CompasDataset, AdultDataset

from sklearn.metrics import f1_score
from functions import*

import os
path=os.path.dirname(os.getcwd())

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

"""
# if you need "OptimPreproc"
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions\
            import get_distortion_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
"""

c:\Users\zhouq\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


'\n# if you need "OptimPreproc"\nfrom aif360.algorithms.preprocessing.optim_preproc import OptimPreproc\nfrom aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions            import get_distortion_adult\nfrom aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools\n'

In [2]:
class Projpreprocess:
    
    def __init__(self,traindata,x_list,var_list,K,e):

        self.K=K
        self.e=e
        self.x_list=x_list
        self.var_list=var_list
      
        self.var_dim=len(var_list)
        self.arg_list=[elem for elem in var_list if elem not in x_list]
        self.train = traindata.copy()
        self.df = self.train.convert_to_dataframe()[0]
        self.pa = self.train.protected_attribute_names[0]
        self.pa_index = self.train.feature_names.index(pa)
        self.label_name = self.train.label_names[0]
        self.df=self.df.rename(columns={self.pa:'S',self.label_name:'Y'})

        self.df['W'] = self.train.instance_weights
        for col in self.var_list+['S','Y']:
            self.df[col]=self.df[col].astype('int64')
        self.df=self.df[var_list+['S','W','Y']]
        if len(x_list)>1:
            self.df['X'] = list(zip(*[self.df[c] for c in x_list]))
            self.x_range=sorted(set(self.df['X']))
            weight=list(1/(self.df[x_list].max()-self.df[x_list].min())) # because ranges of attributes differ
            self.C=c_generate_higher(self.x_range,weight)
        else:
            self.df['X']=self.df[x_list]
            self.x_range=sorted(set(self.df['X']))
            self.C=c_generate(self.x_range)
        self.df = self.df[self.arg_list+['X','S','Y','W']].groupby(by=self.arg_list+['X','S','Y'],as_index=False).sum()
        self.distribution_generator()
        
    def distribution_generator(self):
        bin=len(self.x_range)
        dist=rdata_analysis(self.df,self.x_range,'X')
        dist['v']=[(dist['x_0'][i]-dist['x_1'][i])/dist['x'][i] for i in range(bin)]
        dist['t_x']=dist['x'] # #dist['x'] #dist['x_0']*0.5+dist['x_1']*0.5 
        self.px=np.matrix(dist['x']).T
        self.ptx=np.matrix(dist['t_x']).T
        if np.any(dist['x_0']==0): 
            self.p0=np.matrix((dist['x_0']+1.0e-9)/sum(dist['x_0']+1.0e-9)).T
        else:
            self.p0=np.matrix(dist['x_0']).T 
        if np.any(dist['x_1']==0):
            self.p1=np.matrix((dist['x_1']+1.0e-9)/sum(dist['x_1']+1.0e-9)).T
        else:
            self.p1=np.matrix(dist['x_1']).T 
        self.V=np.matrix(dist['v']).T
        # self.tv_origin=sum(abs(dist['x_0']-dist['x_1']))/2
        # return px,ptx,V,p0,p1
    
    def coupling_generator(self,method,Theta=1e-2):
        print(Theta)
        if method == 'unconstrained':
            coupling=baseline(self.C,self.e,self.px,self.ptx,self.K)
        elif method == 'barycentre':
            coupling=baseline(self.C,self.e,self.p0,self.p1,self.K)
        elif method == 'partial':
            coupling=partial_repair(self.C,self.e,self.px,self.ptx,self.V,Theta,self.K)
        return coupling

    def preprocess(self,method,Theta=1e-2):
        coupling = self.coupling_generator(method,Theta)
        if len(self.x_list)>1:
            df_proj=projection_higher(self.df,coupling,self.x_range,self.x_list,self.var_list)
        else:
            df_proj=projection(self.df,coupling,self.x_range,self.x_list[0],self.var_list)
        df_proj = df_proj.groupby(by=self.arg_list+['X','S','Y'],as_index=False).sum()
        X=list(zip(*df_proj['X']))
        df_proj = df_proj.assign(**{self.x_list[i]:X[i] for i in range(len(self.x_list))})
        df_proj=df_proj.drop('X',axis=1)
        df_proj=df_proj.rename(columns={'S':self.pa,'Y':self.label_name})
        binaryLabelDataset = BinaryLabelDataset(
            favorable_label=0,
            unfavorable_label=1,
            df=df_proj.drop('W',axis=1), 
            label_names=self.train.label_names,
            protected_attribute_names=self.train.protected_attribute_names,
            privileged_protected_attributes=[np.array([1.0])],unprivileged_protected_attributes=[np.array([0.])])
        binaryLabelDataset.instance_weights = df_proj['W'].tolist()
        # return binaryLabelDataset.align_datasets(self.train)
        return self.train.align_datasets(binaryLabelDataset)

In [3]:
class Baselinepreprocess:

    def __init__(self,train,test):
        self.train = train
        self.test = test
        self.pa = train.protected_attribute_names[0]
        self.pa_index = train.feature_names.index(pa)
        self.prigroups = [{self.pa: 1}]
        self.unprigroups = [{self.pa: 0}]

    def preprocessing(self,method):
        test_tranf = self.test.copy()
        if method == 'RW':
            RW = Reweighing(privileged_groups = self.prigroups,unprivileged_groups = self.unprigroups) #DisparateImpactRemover(repair_level = 1)
            RW.fit(self.train)
            train_tranf = RW.transform(self.train)
        elif method == 'DIremover':
            di = DisparateImpactRemover(repair_level = 1,sensitive_attribute=pa)
            train_tranf = di.fit_transform(self.train)
            test_tranf = di.fit_transform(self.test)
        elif method == 'LFR':
            TR = LFR(privileged_groups = self.prigroups,unprivileged_groups = self.unprigroups,
                     Az = 1, Ax = 0.01, Ay = 1,verbose=0)
            TR = TR.fit(self.train)
            train_tranf = TR.transform(self.train)
            test_tranf = TR.transform(self.test)
        elif method == 'OP':
            optim_options = {
                "distortion_fun": get_distortion_adult,
                "epsilon": 0.05,
                "clist": [0.99, 1.99, 2.99],
                "dlist": [.1, 0.05, 0]
            }
            OP = OptimPreproc(OptTools, optim_options)
            OP = OP.fit(self.train)
            train_tranf = OP.transform(self.train, transform_Y=True)
        return train_tranf, test_tranf

    def prediction(self,method,para=None):
        test_tranf = self.test.copy()
        if method == 'origin':
            train_tranf = self.train
        elif method in ['RW','DIremover','LFR','OP']:
            train_tranf,test_tranf = self.preprocessing(method)
        else:
            K=200
            e=0.01
            var_list=self.train.feature_names.copy()
            var_list.remove(self.pa)
            projpre=Projpreprocess(self.train,para['x_list'],var_list,K,e)
            train_tranf=projpre.preprocess(method,para['Theta'])

        di=self.DisparateImpact(train_tranf)
        print('Disparate Impact of train',di)

        if method != 'LFR':
            X_train = np.delete(train_tranf.features, self.pa_index, axis=1)
            y_train = train_tranf.labels.ravel()
            weight_train = train_tranf.instance_weights
            model=RandomForestClassifier(max_depth=5).fit(X_train,y_train, sample_weight=weight_train)

            X_test = np.delete(test_tranf.features, self.pa_index, axis=1)
            y_pred = model.predict(X_test)
        else:
            y_pred = test_tranf.labels
        return y_pred,di
    
    def DisparateImpact(self,data):
        di = pd.DataFrame({'S':data.protected_attributes.ravel().tolist(),
            'Y':data.labels.ravel().tolist(),
            'W':list(data.instance_weights)},columns=['S','Y','W'])
        privileged = self.train.privileged_protected_attributes[0][0]
        unprivileged = self.train.unprivileged_protected_attributes[0][0]
        numerator=sum(di[(di['S']==unprivileged)&(di['Y']==data.favorable_label)]['W'])/sum(di[di['S']==unprivileged]['W'])
        denominator=sum(di[(di['S']==privileged)&(di['Y']==data.favorable_label)]['W'])/sum(di[di['S']==privileged]['W'])
        if numerator==denominator:
            return 1
        return numerator/denominator

    def assess(self,method,para=None):
        if para != None:
            y_pred,di_train = self.prediction(method,para)
        else:
            y_pred,di_train = self.prediction(method)
        y_test_pred = self.test.copy()
        y_test_pred.labels = y_pred

        di=self.DisparateImpact(y_test_pred)
        f1_macro = f1_score(self.test.labels, y_pred, average='macro',sample_weight=self.test.instance_weights)
        f1_micro = f1_score(self.test.labels, y_pred, average='micro',sample_weight=self.test.instance_weights)
        f1_weighted = f1_score(self.test.labels, y_pred, average='weighted',sample_weight=self.test.instance_weights)
        print('Disparate Impact of '+str(method),di)
        print('f1 macro of '+str(method),f1_macro)

        new_row=pd.Series({'DI of train':di_train,'DI':di,'f1 macro':f1_macro,'f1 micro':f1_micro,'f1 weighted':f1_weighted,'method':method})
        return new_row.to_frame().T

# Compas

In [4]:
pa = 'race'
label_map = {1.0: 'Did recid.', 0.0: 'No recid.'}
protected_attribute_maps = {1.0: 'Caucasian', 0.0: 'Not Caucasian'}
privileged_groups = [{pa: 1}]
unprivileged_groups = [{pa: 0}]
cd = CompasDataset(protected_attribute_names=[pa],privileged_classes=[['Caucasian'],[1]], 
                    metadata={'label_map': label_map,'protected_attribute_maps': protected_attribute_maps},
                    features_to_drop=['age', 'sex', 'c_charge_desc'])
train,test = cd.split([0.6], shuffle=True) #len(test.instance_names) = 2057
var_list = cd.feature_names.copy()
var_list.remove(pa)
var_dim=len(var_list)

# df_train = df.loc[train.instance_names,:].reset_index(drop=True)
# df_test = df.loc[test.instance_names,:].reset_index(drop=True)
df=cd.convert_to_dataframe()[0]
df=df.rename(columns={pa:'S',cd.label_names[0]:'Y'})
df['W'] = cd.instance_weights
for col in var_list+['S','Y']:
    df[col]=df[col].astype('int64')
df=df[var_list+['S','W','Y']]

tv_dist=dict()
for x_name in var_list:
    x_range_single=list(pd.pivot_table(df,index=x_name,values=['W'],observed=False)[('W')].index) 
    dist=rdata_analysis(df,x_range_single,x_name)
    tv_dist[x_name]=sum(abs(dist['x_0']-dist['x_1']))/2
    print(x_name, tv_dist[x_name])
x_list=[]
for key,val in tv_dist.items():
    if val>0.1:
        x_list+=[key]

juv_fel_count 0.03210337325453563
juv_misd_count 0.04323143324022939
juv_other_count 0.021763780679615215
priors_count 0.12622233191661625
age_cat=25 - 45 0.054431947619680315
age_cat=Greater than 45 0.13519019921101838
age_cat=Less than 25 0.08075825159133806
c_charge_degree=F 0.07840757396162046
c_charge_degree=M 0.07840757396162046


In [5]:
methods=['origin','RW','DIremover','LFR'] 
report=pd.DataFrame(columns=['DI of train','DI','f1 macro','f1 micro','f1 weighted','method'])
for ignore in range(10):
    # train val test 4:2:4
    train,test = cd.split([0.4], shuffle=True) 
    valid,test = test.split([0.3], shuffle=True)
    
    prepro = Baselinepreprocess(train,test)
    for method in methods:
        report = pd.concat([report,prepro.assess(method)], ignore_index=True)

report.to_csv(path+'/data/report_preprocess_compas_'+str(pa)+'.csv',index=None)

Disparate Impact of train 0.851899228937924
Disparate Impact of origin 0.7699728647602421
f1 macro of origin 0.6626255228489806
Disparate Impact of train 1.0000000000000047
Disparate Impact of RW 0.7693019448067092
f1 macro of RW 0.6636377580675187
Disparate Impact of train 0.851899228937924
Disparate Impact of DIremover 0.8303539438551617
f1 macro of DIremover 0.6626644007234119


KeyboardInterrupt: 

# Adult

In [7]:
def load_data(data_path,var_list,pa):
    column_names = ['age', 'workclass', 'fnlwgt', 'education',
                'education-num', 'marital-status', 'occupation', 'relationship',
                'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
                'native-country', 'Y']
    na_values=['?']
    pa_dict={'Male':1,'Female':0,'White':1,'Black':0}
    label_dict={'>50K.':1,'>50K':1,'<=50K.':0,'<=50K':0}
    train_path = os.path.join(data_path, 'adult.data')
    test_path = os.path.join(data_path, 'adult.test')
    train = pd.read_csv(train_path, header=None,names=column_names,
                    skipinitialspace=True, na_values=na_values)
    test = pd.read_csv(test_path, header=0,names=column_names,
                    skipinitialspace=True, na_values=na_values)
    messydata = pd.concat([test, train], ignore_index=True)[var_list+[pa,'Y']]
    messydata=messydata.rename(columns={pa:'S'})
    messydata['S']=messydata['S'].replace(pa_dict)
    messydata['Y']=messydata['Y'].replace(label_dict)
    messydata=messydata[(messydata['S']==0)|(messydata['S']==1)]
    for col in var_list+['S','Y']:
        messydata[col]=messydata[col].astype('int64')
    messydata['W']=1
    bins_capitalgain=[100,3500,7500,10000]
    bins_capitalloss=[100,1600,1900,2200]
    bins_age=[26,36,46,56]
    bins_hours=[21,36,46,61]

    messydata=categerise(messydata,'age',bins_age)
    messydata=categerise(messydata,'hours-per-week',bins_hours)
    messydata=categerise(messydata,'capital-gain',bins_capitalgain)
    messydata=categerise(messydata,'capital-loss',bins_capitalloss)
    
    return messydata

def categerise(df,col,bins):
    for i in range(len(bins)+1):
        if i == 0:
            df.loc[df[col] < bins[i], col] = i
        elif i == len(bins):
            df.loc[df[col] >= bins[i-1], col] = i
        else:
            df.loc[(df[col] >= bins[i-1])& (df[col] < bins[i]), col] = i        
    return df

def choose_x(var_list,messydata):
    tv_dist=dict()
    for x_name in var_list:
        x_range_single=list(pd.pivot_table(messydata,index=x_name,values=['W'])[('W')].index) 
        dist=rdata_analysis(messydata,x_range_single,x_name)
        tv_dist[x_name]=sum(abs(dist['x_0']-dist['x_1']))/2
    x_list=[]
    for key,val in tv_dist.items():
        if val>0.1:
            x_list+=[key]  
    return x_list,tv_dist

In [8]:
data_path='C://Users//zhouq//anaconda3//Lib//site-packages//aif360//data//raw//adult'
var_list=['hours-per-week','age','capital-gain','capital-loss','education-num'] #,'education-num'
pa='race'
favorable_label = 1
var_dim=len(var_list)

messydata = load_data(data_path,var_list,pa)
x_list,tv_dist = choose_x(var_list,messydata)
messydata=messydata.rename(columns={'S':pa})
cd=BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=messydata,label_names='Y',protected_attribute_names=[pa])
train,test = cd.split([0.4], shuffle=True) 
valid,test = test.split([0.3], shuffle=True)

In [9]:
# para={'x_list':x_list,'Theta':1e-2}
methods=['origin','RW','DIremover','LFR'] 
report=pd.DataFrame(columns=['DI of train','DI','f1 macro','f1 micro','f1 weighted','method'])
for ignore in range(10):
    # train val test 4:2:4
    train,test = cd.split([0.4], shuffle=True) 
    valid,test = test.split([0.3], shuffle=True)
    
    prepro = Baselinepreprocess(train,test)
    for method in methods:
        report = pd.concat([report,prepro.assess(method)], ignore_index=True)

report.to_csv(path+'/data/report_preprocess_adult_'+str(pa)+'.csv',index=None)

Disparate Impact of train 0.44873006584429503
Disparate Impact of origin 0.43310047558787607
f1 macro of origin 0.6803595734532597
Disparate Impact of train 0.9999999999996648
Disparate Impact of RW 0.41737106844093386
f1 macro of RW 0.6802187213704387
Disparate Impact of train 0.44873006584429503
Disparate Impact of DIremover 0.43175662520547814
f1 macro of DIremover 0.680426647120933
Disparate Impact of train 0.7084612678238899
Disparate Impact of LFR 0.7413488495107114
f1 macro of LFR 0.6931194934576863
Disparate Impact of train 0.49076993064870994
Disparate Impact of origin 0.4211895340927599
f1 macro of origin 0.6737661346899992
Disparate Impact of train 1.0000000000001794
Disparate Impact of RW 0.4243314278525547
f1 macro of RW 0.6724272320532674
Disparate Impact of train 0.49076993064870994
Disparate Impact of DIremover 0.41475631265337964
f1 macro of DIremover 0.6640466603830236
Disparate Impact of train 0.8544432889769877
Disparate Impact of LFR 0.9562705441196477
f1 macro of 

In [10]:
report

,DI of train,DI,f1 macro,f1 micro,f1 weighted,method
0,0.44873,0.4331,0.68036,0.81588,0.78923,origin
1,1.0,0.417371,0.680219,0.816034,0.789232,RW
2,0.44873,0.431757,0.680427,0.816239,0.789403,DIremover
3,0.708461,0.741349,0.693119,0.810703,0.792485,LFR
4,0.49077,0.42119,0.673766,0.81383,0.784903,origin
5,1.0,0.424331,0.672427,0.813881,0.784343,RW
6,0.49077,0.414756,0.664047,0.811779,0.779873,DIremover
7,0.854443,0.956271,0.68958,0.810498,0.790308,LFR
8,0.45268,0.445982,0.682052,0.818494,0.790191,origin
9,1.0,0.460476,0.686415,0.819724,0.79257,RW


In [235]:
report.to_csv(path+'/data/report_preprocess_compas_'+str(pa)+'_'+str(para['Theta'])+'.csv',index=None)

In [215]:
Baselinepreprocess(train,test).assess('partial',para=para)

0.01
Disparate Impact of train 0.8636921817437819
Disparate Impact of partial 0.8029550791375681
f1 macro of partial 0.6464155528554071


In [216]:
Baselinepreprocess(train,test).assess('LFR')

Disparate Impact of train 1.0
Disparate Impact of LFR 1.0
f1 macro of LFR 0.543134872417983


In [217]:
Baselinepreprocess(train,test).assess('RW')

Disparate Impact of train 1.000000000000009
Disparate Impact of RW 0.7882558927630092
f1 macro of RW 0.6618063993519644


In [218]:
Baselinepreprocess(train,test).assess('DIremover')

Disparate Impact of train 0.8636921817437576
Disparate Impact of DIremover 0.859092543994032
f1 macro of DIremover 0.6634264884568651


In [219]:
Baselinepreprocess(train,test).assess('origin')

Disparate Impact of train 0.8636921817437576
Disparate Impact of origin 0.7855305825785532
f1 macro of origin 0.6626164439044148
